# Deriving Term Frequency from CWB Weather Summary

這篇筆記要示範自然語言處理（[Natural Language Processing](https://en.wikipedia.org/wiki/Natural_language_processing), NLP）中的基本工具：計算 [term frequency](https://en.wikipedia.org/wiki/Tf%E2%80%93idf)。我們想要從氣象局每天的[「天氣概況」](https://www.cwb.gov.tw/V8/C/W/index.html)裡，找出氣象局專家群常常提及的一些台灣及鄰近區域的天氣現象。

以下為氣象局天氣概況的一則範例：

***
```
07fW01136
中央氣象局氣象報告
１１０年３月１３日１１時０分發布
３月１３日８時天氣概況：
一、高氣壓１０２６百帕，在北緯３２度，東經１２０度，即在江蘇，向東緩慢移動。
二、今、明（１３日、１４日）兩天東北季風影響，北部及東北部天氣較涼，其他地區早晚亦涼；臺灣東半部地區有局部短暫雨，其他地區及澎湖、金門、馬祖為多雲到晴，今日午後新竹以南山區亦有局部短暫陣雨，明日新竹以南山區亦有零星短暫雨；今、明兩天桃園至臺南、東南部（含蘭嶼、綠島）、恆春半島沿海空曠地區及澎湖、金門易有較強陣風；明日西半部地區易有局部霧或低雲影響能見度，請注意。
三、海上強風特報：
１、東北風偏強，臺灣海峽北部平均風力６至７級，最大陣風９級；臺灣北部海面、臺灣東南部海面及臺灣海峽南部平均風力可達６級，最大陣風８級，船隻請特別注意。今（１３）日臺灣東南部海面及臺灣海峽平均風力將稍減弱。明（１４日）下午起臺灣東南部海面平均風力將增強至６級，最大陣風８級，船隻請注意。
２、東北風偏強，巴士海峽、廣東海面及南海平均風力可達６級，雷雨區最大陣風８至９級，船隻請注意。今（１３日）晚起中西沙島海面平均風力將增強至６到７級，最大陣風９級，船隻請特別注意。今（１３）日廣東海面平均風力將稍減弱。

```
***

我們希望能從內容裡中找出像是「高氣壓」、「東北季風」、「局部霧」、「低雲」、以及「強風特報」、「颱風警報」這類指涉特定天氣系統或現象的詞彙，並透過分析長時間累積的文本資料，計算各個詞彙出現的頻率，來看看中央氣象局最常提到的天氣現象有哪些。


## 1. 基本資料處理

要計算詞頻，我們需要天氣報告檔案的清單、讀取，以及內容解析的工具，讓我們先進行基本資料的處理。

In [1]:
# 列出測試資料檔
import os, re
import pandas as pd

DATAPATH = '../data/cwb/weather_report/'
SURFIX = '_1100.W01.dat'

fileinfo = []
for root, dirs, files in os.walk(DATAPATH):
    for name in files:
        date = name.replace(SURFIX, '')
        furi = (os.path.join(root, name))
        fileinfo.append({'date':date, 'uri': furi})

# 測試
fileinfo = pd.DataFrame(fileinfo)
print(fileinfo.head())

         date                                                uri
0  2016-01-01  ../data/cwb/weather_report/2016-01-01_1100.W01...
1  2016-02-01  ../data/cwb/weather_report/2016-02-01_1100.W01...
2  2016-03-01  ../data/cwb/weather_report/2016-03-01_1100.W01...
3  2016-04-01  ../data/cwb/weather_report/2016-04-01_1100.W01...
4  2016-05-01  ../data/cwb/weather_report/2016-05-01_1100.W01...


### 檔名與路徑

我們除了把檔名列出來，也把日期擷取出來，這樣方便我們之後可以針對不同年份、月份和日期做分析。接下來我們需要讀取單一檔案的工具。

In [2]:
# 讀取單一檔案
def read_cwb_summary(furi):
    import os, re
    # Read file content
    with open(furi, 'r', encoding='utf8') as f:
        text = f.readlines()
    text = ''.join(text)
    return(text)

# 測試
data = read_cwb_summary(fileinfo['uri'].iloc[0])
print(data[:100])

氣象報告
１０５年１月１日１１時０分發布
１月１日８時天氣概況：

　　高氣壓１０３６百帕，在北緯３２度，東經１２３度，即
在東海北部海面，向東移動，時速１５公里。

　　今、明（１日、２日）兩天東北


### 內容解析

要計算詞頻，需要先把特定的詞彙找出來，要達到這個目的有幾種作法，主要區分為 top-dwon 跟 bottom-up 兩個方向。

**Top-down approach** 是我們已經有定義好的詞彙清單，然後在每篇文本裡搜尋，找到的就+1。這種方法簡單快速，但是有一個很基本的缺點，就是這個「事先定義好的詞彙清單」在文本裡必須被嚴格精確的遵循，不能有任何例外。一般對於嚴格格式的文本才能用這種方式，對於自由文本來說，一方面這個清單很難事先知道，二方面同一個詞彙可能存在許多例外，所以不見得適用。

**Bottom-up approach** 是先將文本分割為詞彙的組合，然後計算所有詞彙出現的次數。這個方法完全的 data-driven，但是在一般文本中通常出現次數最多的詞彙會是標點符號、指示代名詞跟各種語助詞，經驗上我們有興趣的詞彙通常出現在中間段，但是究竟在哪裡實在不容易事先得知，而這也是這個方法的缺點：需要反覆的嘗試，才能得到容易詮釋的結果。

實作上，常常是兩種方法的組合：我們先以 **bottom-up approach**， 找出我們感興趣的詞彙（以及各自的同義詞），然後再以 **top-down approach** 去計算詞彙發生的次數，這個過程通常要反覆數次，才會有比較容易詮釋的結果。


### 中文斷詞

語言學上，將文本分割成分析單位的過程叫做標記化（[tokenization](https://en.wikipedia.org/wiki/Lexical_analysis#Tokenization)），中文處理上比較簡單的方法是利用現有的中文斷詞（[word segmentation](https://en.wikipedia.org/wiki/Text_segmentation#Word_segmentation)）工具，我們在下面的例子裡使用目前最常用的 [jieba](https://github.com/fxsjy/jieba) 跟 [CKIP tagger](https://github.com/ckiplab/ckiptagger) 來計算詞頻。

In [21]:
# 利用 jieba 分詞
def calculate_tf_with_jieba(article, sort=False):
    import jieba
    # Perform word segmentation
    seg_list = jieba.cut(data, cut_all=False)
    # Count word frequency
    wc={}
    for w in seg_list:
        if not w in  wc:
            wc[w] = 1
        else:
            wc[w] += 1
    if sort:
        wc = dict(sorted(wc.items(), key=lambda item: item[1], reverse=True))
    return(wc)

wc = calculate_tf_with_jieba(data)
print(wc)

{'氣象': 1, '報告': 1, '\n': 40, '１': 21, '０': 9, '９': 6, '年': 1, '２': 8, '月': 2, '日': 9, '時': 1, '分發布': 1, '８': 10, '時天氣': 1, '概況': 1, '：': 3, '\u3000': 10, '天氣': 2, '特報': 3, '請': 10, '參照': 1, '本局': 1, '發布': 1, '之': 1, '最新': 1, '豪': 1, '(': 1, '大': 3, ')': 1, '雨特報': 1, '、': 22, '陸上': 1, '強風': 2, '。': 13, '高氣壓': 1, '５': 2, '百帕': 1, '，': 34, '在': 2, '北緯': 1, '４': 2, '度': 2, '東經': 1, '即': 1, '蒙古': 1, '向': 1, '東南東': 1, '緩慢': 1, '移動': 1, '今': 4, '明': 4, '（': 9, '）': 9, '兩天': 1, '東北季風': 1, '影響': 2, '北部': 8, '東': 8, '及': 10, '東部': 2, '較': 2, '涼': 2, '其他': 2, '地區': 5, '早晚': 1, '亦': 1, '中南部': 1, '日夜': 1, '溫差': 1, '；': 7, '基隆': 2, '北海岸': 2, '臺灣': 5, '臺': 2, '北山': 1, '區有': 1, '雨': 1, '南部': 10, '恆春半島': 3, '有': 2, '局部': 1, '短': 1, '暫雨': 1, '澎湖': 2, '金': 1, '門': 1, '馬': 2, '祖為': 1, '多雲到': 1, '晴': 1, '明天': 1, '南': 1, '以北': 1, '半部': 2, '含蘭嶼': 1, '綠島': 1, '沿海': 1, '空曠': 1, '金門': 1, '祖易': 1, '至': 13, '級': 27, '強陣風': 1, '含綠島': 1, '蘭嶼': 1, '有長': 1, '浪': 1, '發生': 1, '的': 1, '機率': 1, '注意': 9, '海上': 1, '東北風': 2

In [4]:
# 利用 ckiptagger 分詞
def calculate_tf_with_ckiptagger(article, sort=False):
    from ckiptagger import data_utils, construct_dictionary, WS
    ws = WS("D:\worksapce\ckiptagger\data")
    # Perform word segmentation
    seg_list = ws(
                [data.replace('\n','')],
                sentence_segmentation = True, # To consider delimiters
                segment_delimiter_set = {",", "。", ":", "?", "!", ";"})
    #print(seg_list[0])
    # Count word frequency
    wc={}
    for w in seg_list[0]:
        if not w in  wc:
            wc[w] = 1
        else:
            wc[w] += 1
    if sort:
        wc = dict(sorted(wc.items(), key=lambda item: item[1], reverse=True))
    return(wc)

wc = calculate_tf_with_ckiptagger(data)
print(wc)


{'氣象': 1, '報告': 1, '１０５年': 1, '１月': 2, '１日': 4, '１１時': 1, '０分': 1, '發布': 1, '８時': 1, '天氣': 1, '概況': 1, '：': 2, '\u3000\u3000': 3, '高氣壓': 1, '１０３６': 1, '百帕': 1, '，': 22, '在': 2, '北緯': 1, '３２': 1, '度': 2, '東經': 1, '１２３': 1, '即': 1, '東海': 2, '北部': 5, '海面': 13, '向': 1, '東': 4, '移動': 1, '時速': 1, '１５': 1, '公里': 1, '。': 10, '今': 3, '、': 15, '明': 4, '（': 3, '２日': 2, '）': 3, '兩': 1, '天': 1, '東北': 1, '季風': 1, '稍': 4, '減弱': 4, '氣溫': 1, '逐漸': 1, '回升': 1, '臺灣': 6, '各': 4, '地': 1, '及': 10, '澎湖': 2, '金門': 2, '馬祖': 2, '大多': 1, '為': 1, '多': 1, '雲': 1, '到': 2, '晴': 1, '僅': 1, '半': 3, '部': 3, '地區': 3, '山區': 1, '有': 4, '局部': 1, '短暫': 1, '雨': 1, '臺南': 1, '以北': 1, '恆春': 2, '半島': 2, '蘭嶼': 1, '綠島': 1, '空曠': 1, '鄰近': 1, '海域': 1, '仍': 1, '較': 1, '強': 1, '陣風': 6, '；': 3, '沿海': 1, '長浪': 1, '發生': 1, '的': 1, '機率': 1, '請': 5, '注意': 5, '海': 1, '上': 1, '強風': 1, '特報': 1, '１': 1, '東北風': 2, '影響': 3, '臺灣海峽': 1, '平均': 8, '風力': 8, '６': 5, '至': 3, '７': 3, '級': 10, '最': 5, '大': 5, '９': 3, '附近': 2, '其他': 1, '可': 2, '達': 2, '８'

從上面的例子來看，中研院開發的 [CKIP tagger](https://github.com/ckiplab/ckiptagger) 對於專有名詞的辨識能力較佳，但是在實際使用上需要下載一個 [2Gb 大小的模型](https://drive.google.com/drive/folders/105IKCb88evUyLKlLondvDBoh7Dy_I1tm)，安裝程序相對比較複雜。而 [jieba](https://github.com/fxsjy/jieba) 在安裝與使用上較為便利，專有名詞辨識能力不足的缺點，則需要透過另外提供自行定義的辭典來克服。實際上在分析大量文本的時候，[CKIP tagger](https://github.com/ckiplab/ckiptagger) 也需要透過自行定義的辭典來輔助，而且運算時間較常的缺點也很明顯，因此可以自行斟酌使用。

以下我們用兩個方法分析60個檔案，看看結果有什麼異同。

In [5]:
def merge_dictionary(dict1, dict2, sort=False):
    from collections import Counter
    wc = Counter(dict1) + Counter(dict2)
    if sort:
        wc = dict(sorted(wc.items(), key=lambda item: item[1], reverse=True))
    return(wc)

wc_jieba = {}
wc_ckip = {}
for i in range(fileinfo.shape[0]):
    #print(fileinfo['uri'].iloc[i])
    data = read_cwb_summary(fileinfo['uri'].iloc[i])
    #
    tmp_jieba = calculate_tf_with_jieba(data)
    #tmp_ckip = calculate_tf_with_ckiptagger(data)
    #
    wc_jieba = merge_dictionary(wc_jieba, tmp_jieba)
    #wc_ckip = merge_dictionary(wc_ckip, tmp_ckip)


In [6]:
# Results of jieba
print(wc_jieba)

Counter({'\n': 2221, '，': 1925, '１': 1287, '級': 914, '海面': 759, '、': 752, '。': 658, '２': 594, '０': 587, '（': 563, '）': 563, '\u3000': 556, '平均': 532, '日': 513, '臺': 501, '８': 457, '及': 449, '６': 448, '最大': 447, '陣風': 412, '請': 392, '至': 389, '北部': 385, '注意': 333, '９': 330, '風力將': 312, '南部': 310, '７': 299, '隻': 298, '船': 297, '；': 248, '度': 246, '明': 232, '地區': 227, '增強': 224, '風力': 220, '有': 219, '３': 217, '在': 206, '今': 206, '公里': 187, '灣': 183, '雷雨': 172, '：': 165, '局部': 162, '短': 155, '灣東': 155, '影響': 143, '特別': 142, '５': 140, '即': 135, '的': 135, '到': 131, '減弱': 129, '稍': 128, '區': 127, '月': 120, '風': 120, '東經': 119, '馬': 118, '４': 116, '北緯': 115, '灣海峽': 115, '金門': 114, '島': 113, '澎湖': 109, '特報': 105, '時': 99, '之': 97, '可達': 96, '東': 94, '下午': 94, '半部': 93, '東海': 90, '海峽': 90, '巴士': 88, '中心': 88, '百帕': 81, '為': 80, '．': 80, '暫陣雨': 77, '天氣': 74, '強風': 73, '其他': 73, '上': 71, '中': 68, '西沙': 67, '或': 66, '颱': 65, '浙江': 63, '晴': 62, '氣象': 61, '年': 61, '報告': 60, '分發布': 60, '時天氣': 60, '概況'

In [31]:
# Results of ckip
print(wc_ckip)

{'，': 1925, '級': 949, '海面': 784, '、': 752, '。': 658, '及': 599, '平均': 555, '風力': 555, '大': 524, '最': 505, '陣風': 459, '至': 430, '請': 396, '將': 395, '（': 374, '）': 374, '有': 349, '臺灣': 347, '注意': 346, '６': 332, '船隻': 298, '北部': 289, '\u3000\u3000': 277, '１日': 260, '明': 255, '地區': 255, '；': 248, '度': 243, '增強': 243, '８': 227, '９': 226, '７': 209, '在': 206, '起': 205, '今': 203, '公里': 192, '到': 190, '日': 188, '影響': 177, '局部': 168, '短暫': 166, '：': 165, '南部': 162, '特別': 156, '１０': 146, '上': 144, '雷雨區': 140, '稍': 140, '２日': 140, '天氣': 138, '減弱': 136, '的': 135, '即': 135, '東南部': 130, '馬祖': 128, '特報': 126, '臺灣海峽': 125, '金門': 122, '北緯': 120, '東經': 120, '（２）': 119, '澎湖': 116, '東海': 115, '東北部': 106, '可': 105, '達': 105, '之': 100, '半': 98, '部': 98, '下午': 98, '８時': 96, '陣雨': 91, '中心': 91, '東': 91, '每': 90, '巴士海峽': 89, '發布': 87, '其他': 84, '百帕': 81, '雨': 80, '為': 79, '向': 79, '雲': 78, '多': 77, '各': 74, '廣東': 74, '東沙島': 73, '強風': 73, '（１）': 70, '或': 67, '浙江': 67, '中西沙島': 67, '颱風': 65, '東北風': 65, '風': 64, '發生

從以上的結果來看， CKIP tagger 的斷詞效果雖然比較好，對於地名、位置、方向位置的描述斷詞的比較精確，但是對於像是「東北季風」、「溫帶氣旋」這些我們關心的特殊詞彙，依然被斷成兩個詞，需要額外自行定義辭典的輔助。

在計算時間上，每次使用 CKIP tagger 都需要載入一次 tensorflow 的模型，相較於 jieba 來說慢上很多。以進行一次斷詞來說，jieba:<1sec, CKIP tagger: ~10sec，當我們對3000個檔案進行斷詞時，這個時間差距會非常大。

綜合以上的觀察，我們後續將使用 jieba 配合自定義辭典進行分析。


### 加入自定義辭典和排除特定詞彙

從前面的初步結果當中，我們可以發現**標點符號**、**全形數字**和**分行符號**的出現頻率很高，但是並非我們所關心的「天氣現象」，因此我們可以用一個清單（ignore-list）把這類的詞彙排除在最後的報表之外。以下是搭配了自定義辭典和忽略清單的結果。

In [25]:
# 讀取單一檔案
def read_cwb_summary(furi):
    import os, re
    # Read file content
    with open(furi, 'r', encoding='utf8') as f:
        text = f.readlines()
    text = ''.join(text)
    return(text)

# 利用 jieba 分詞
def calculate_tf_with_jieba(article, userdict='../data/cwb/userdict.txt'):
    import jieba
    jieba.load_userdict(userdict)    # Load user-dictionary
    # Perform word segmentation
    seg_list = jieba.cut(article, cut_all=True)
    # Count word frequency
    wc={}
    for w in list(seg_list):
        if not w in wc:
            wc[w] = 1
        else:
            wc[w] += 1
    return(wc)

# Merge two dicts
def merge_dictionary(dict1, dict2):
    newdict = dict1.copy()
    for k, v in dict2.items():
        #print(k,v)
        if k not in newdict.keys():
            newdict[k]=v
            #print('Add key: '+k)
        else:
            newdict[k]+=v
            #print('Update key: '+k)
    return(newdict)

# 
def count_words(DATAPATH, SURFIX = '_1100.W01.dat', 
                userdict='../data/cwb/userdict.txt', 
                ignorelist='../data/cwb/ignore.txt'):
    import os, re
    import pandas as pd
    import jieba
    from collections import Counter
    # Get files
    fileinfo = []
    for root, dirs, files in os.walk(DATAPATH):
        for name in files:
            date = name.replace(SURFIX, '')
            furi = (os.path.join(root, name))
            fileinfo.append({'date':date, 'uri': furi})
    fileinfo = pd.DataFrame(fileinfo)
    # Perform word segmentation and counts
    data = read_cwb_summary(fileinfo['uri'].iloc[0])
    wc = calculate_tf_with_jieba(data, userdict=userdict)
    for i in range(1,fileinfo.shape[0]):
        #print(fileinfo['uri'].iloc[i])
        data = read_cwb_summary(fileinfo['uri'].iloc[i])
        tmp = calculate_tf_with_jieba(data, userdict=userdict)
        #print(len(tmp))
        wc = merge_dictionary(wc, tmp)
    # Load ignore-list
    with open('../data/cwb/ignore.txt', 'r', encoding='utf8') as f:
        to_ignore = f.read().split('\n')
    to_ignore.append('\n')
    to_ignore.append('\u3000')
    # Remove terms in ignore list
    for term in to_ignore:
        for k in wc.copy().keys():
            if term in k:
                #print(k)
                del wc[k]
    # Sort by counts
    wc = dict(sorted(wc.items(), key=lambda item: item[1], reverse=True))
    # Done
    return(wc)

wc = count_words('../data/cwb/weather_report/')
print(wc)

{'': 3453, '風': 992, '級': 950, '東': 859, '海面': 759, '日': 586, '及': 575, '力': 555, '平均': 532, '臺灣': 457, '最大': 447, '請': 396, '將': 396, '最大陣風': 389, '北部': 385, '至': 381, '強': 351, '南部': 350, '有': 341, '注意': 334, '區': 317, '氣': 308, '船': 298, '隻': 298, '明': 286, '特': 282, '度': 277, '時': 260, '島': 243, '增': 243, '地': 236, '在': 206, '今': 206, '到': 190, '報': 188, '公里': 187, '雨': 183, '北': 182, '天': 180, '影': 177, '響': 177, '雷雨': 172, '短': 168, '暫': 166, '：': 165, '局部': 162, '發': 160, '海': 159, '陣': 157, '別': 156, '稍': 140, '減': 137, '即': 136, '弱': 136, '的': 135, '馬': 134, '雷雨區': 131, '祖': 128, '經': 125, '金': 122, '門': 122, '月': 120, '緯': 120, '起': 120, '海南': 115, '為': 114, '臺灣海峽': 110, '澎湖': 109, '可': 105, '達': 105, '峽': 104, '雲': 101, '之': 100, '向': 99, '下午': 94, '半部': 93, '壓': 88, '巴士': 88, '大': 88, '中心': 88, '布': 87, '帕': 82, '百': 81, '多': 80, '其他': 80, '沙': 78, '高': 77, '廣': 77, '黃': 77, '間': 75, '面上': 68, '西沙': 67, '或': 67, '中西': 65, '颱': 65, '晴': 64, '生': 64, '鳥': 64, '東北風': 63, '浙江':

加入自定義辭典跟排除清單後，結果仍然需要反覆調整，才能得到乾淨的結果。

In [28]:
# Tuning the userdict.txt and ignore.txt
wc = count_words('../data/cwb/weather_report/')
# Remove all unigram
for k in wc.copy().keys():
    if len(k)<=1:
        del wc[k]
# Remove singleton
for k in wc.copy().keys():
    if wc[k]<=1:
        del wc[k]
print(wc)

{'海面': 759, '平均': 532, '臺灣': 457, '最大': 447, '最大陣風': 389, '北部': 385, '南部': 350, '注意': 334, '公里': 187, '雷雨': 172, '局部': 162, '雷雨區': 131, '海南': 115, '臺灣海峽': 110, '澎湖': 109, '下午': 94, '半部': 93, '巴士': 88, '中心': 88, '其他': 80, '面上': 68, '西沙': 67, '中西': 65, '東北風': 63, '浙江': 63, '山海': 62, '江海': 61, '海上': 60, '沿海': 54, '各地': 52, '起至': 52, '西南': 49, '南沙': 48, '晚起': 42, '海北': 41, '附近': 40, '恆春半島': 39, '上午': 39, '鋒面': 37, '最新': 37, '公尺': 35, '位置': 34, '中南': 34, '中南部': 34, '每秒': 33, '大雨': 32, '大多': 31, '以北': 31, '雷陣雨': 30, '低氣壓': 30, '強陣風': 28, '南方': 28, '本局': 27, '零星': 27, '偏南': 24, '明日': 24, '西半': 23, '及其': 23, '中部': 22, '早晚': 22, '梯度': 22, '偏北': 21, '海空': 20, '鵝鑾鼻': 20, '東北季風': 18, '近海': 18, '台海': 18, '命名': 18, '速度': 18, '琉球': 18, '熱帶性低氣壓': 18, '白天': 17, '日本': 17, '海地': 16, '南西': 16, '中文': 16, '西北': 16, '西方': 16, '其外': 16, '今日': 15, '清晨': 14, '日夜': 13, '近似': 13, '豪雨': 13, '北方': 13, '北西': 12, '南海': 12, '延伸': 11, '起花': 11, '海域': 10, '上述': 10, '基隆': 10, '北海': 10, '回升': 9, '北海岸': 9, '海岸': 9, '至上': 9

In [1]:
# 讀取單一檔案
def read_cwb_summary(furi):
    import os, re
    # Read file content
    with open(furi, 'r', encoding='utf8') as f:
        try:
            text = f.readlines()
        except UnicodeDecodeError as e:
            print(furi)
    text = ''.join(text)
    return(text)

# 利用 jieba 分詞
def calculate_tf_with_jieba(article, userdict, char_to_remove):
    import jieba
    jieba.load_userdict(userdict)    # Load user-dictionary
    # Perform word segmentation
    seg_list = jieba.cut(article, cut_all=True)
    # Count word frequency
    wc={}
    for w in list(seg_list):
        if not w in wc:
            wc[w] = 1
        else:
            wc[w] += 1
    # Remove keys containing character to remove
    for term in char_to_remove:
        for k in wc.copy().keys():
            if term in k:
                del wc[k]
    # Done
    return(wc)

# Merge two dicts
def merge_dictionary(dict1, dict2):
    newdict = dict1.copy()
    for k, v in dict2.items():
        #print(k,v)
        if k not in newdict.keys():
            newdict[k]=v
        else:
            newdict[k]+=v
    return(newdict)

# 
def count_words(DATAPATH, SURFIX = '_1100.W01.dat',
                userdict='../data/cwb/userdict.txt', 
                ignorelist='../data/cwb/ignore.txt',
                min_count=1, min_length=1):
    import os, re
    import pandas as pd
    import jieba
    from collections import Counter
    # Get files
    fileinfo = []
    for root, dirs, files in os.walk(DATAPATH):
        for name in files:
            if name.endswith(SURFIX):
                date = name.replace(SURFIX, '')
                furi = (os.path.join(root, name))
                fileinfo.append({'date':date, 'uri': furi})
    fileinfo = pd.DataFrame(fileinfo)
    #print(fileinfo.head())
    # Define characters to remove
    char_to_remove = ['１', '２', '３', '４', '５', '６', '７', '８', '９', '０', 
                      '，', '。', '、', '(', ')', '（', '）', '；', '\n', '\u3000','07fW']
    # Perform word segmentation and counts
    data = read_cwb_summary(fileinfo['uri'].iloc[0])
    wc = calculate_tf_with_jieba(data, userdict, char_to_remove)
    for i in range(1,fileinfo.shape[0]):
        data = read_cwb_summary(fileinfo['uri'].iloc[i])
        tmp = calculate_tf_with_jieba(data, userdict, char_to_remove)
        wc = merge_dictionary(wc, tmp)
    # Load ignore-list
    with open(ignorelist, 'r', encoding='utf8') as f:
        to_ignore = f.read().split('\n')
    to_ignore.append('\n')
    to_ignore.append('\u3000')
    # Remove terms in ignore list
    for term in to_ignore:
        if term in wc.copy().keys():
            del wc[term]
    # Remove terms with length less than min_mlength
    if min_length>1:
        for k in wc.copy().keys():
            if len(k)<min_length:
                del wc[k]
    if min_count>1:
        for k in wc.copy().keys():
            if wc[k]<min_count:
                del wc[k]
    # Sort by counts
    wc = dict(sorted(wc.items(), key=lambda item: item[1], reverse=True))
    # Done
    return(wc)

wc = count_words('../data/cwb/weather_report/', min_count=2, min_length=2)
print(wc)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\tsyo\AppData\Local\Temp\jieba.cache
Loading model cost 0.572 seconds.
Prefix dict has been built successfully.


{'雷雨': 172, '雷雨區': 131, '特報': 126, '颱風': 64, '東北風': 63, '鋒面': 37, '大雨': 32, '雷陣雨': 30, '低氣壓': 30, '強陣風': 28, '東北季風': 18, '熱帶性低氣壓': 18, '大陸冷氣團': 15, '西南風': 14, '豪雨': 13, '菲律賓': 7, '西南氣流': 5, '韓國': 3, '寒流': 3, '華南雲雨區': 2, '玉兔': 2, '淹水': 2}


In [4]:
wc = count_words('D:/data/cwb_report', min_count=20, min_length=2)
print(wc)

{'雷雨': 8242, '雷雨區': 6442, '特報': 5999, '颱風': 2916, '東北風': 2846, '鋒面': 2812, '低氣壓': 1642, '雷陣雨': 1493, '東北季風': 1389, '大雨': 1256, '強陣風': 993, '熱帶性低氣壓': 924, '西南風': 572, '大陸冷氣團': 478, '菲律賓': 369, '豪雨': 289, '馬尼拉': 288, '華南雲雨區': 158, '警報': 153, '寒流': 142, '西南氣流': 108, '韓國': 100, '拉西': 68, '溫帶氣旋': 57, '降雨': 49, '威克': 36, '安全': 35, '注意安全': 31, '天寒': 30, '雅浦': 21, '雅浦島': 21, '哈隆': 20}


## The Purely Bottom-up approach: n-gram

前面的分析使用了「斷詞」工具當做起點，在一般的應用情境下是不錯的選擇，但是斷詞工具難免會把詞彙斷在不應該斷的地方，以致於某部分特定的詞彙可能被忽略掉，尤其是只存在於特定文本而非一般日常詞彙的用語。如果我們要分析的文本跟日常用語差異比較大（例如某地方言或某個特殊族群的用語），希望不要錯過這些詞彙，則也可以考慮使用純粹 bottom-up 的方法：[n-gram](https://en.wikipedia.org/wiki/N-gram)。

我們不準備在這裡介紹 n-gram 的理論細節，只是純粹的使用 n-gram 的概念，在我們的應用情境裡就是把所有「**連續出現的字**」都當做潛在的「**詞彙**」，然後利用大量文本裡出現的次數，自動把「詞彙」跟「非詞彙」區分開來。

In [29]:
from collections import Counter
from nltk import FreqDist
from nltk.util import ngrams, everygrams

# 讀取單一檔案
def read_cwb_summary(furi):
    import os, re
    # Read file content
    with open(furi, 'r', encoding='utf8') as f:
        try:
            text = f.readlines()
        except UnicodeDecodeError as e:
            print(furi)
    text = ''.join(text)
    return(text)

def compute_freq(uri, min_n=2, max_n=6):
    stopwords = ['１', '２', '３', '４', '５', '６', '７', '８', '９', '０', 
                 '，', '。', '、', '(', ')', '（', '）', '；', '\n', '\u3000','07fW']
    textfile = read_cwb_summary(uri)

    ngrams = everygrams(textfile, 2, 6)
    clean = [''.join(gram) for gram in ngrams if not any(stop in gram for stop in stopwords)]
    
    return(Counter(clean))

def count_ngrams(DATAPATH, SURFIX = '_1100.W01.dat', min_count=1, min_n=2, max_n=6):
    import os, re
    import pandas as pd
    from nltk import FreqDist
    from nltk.util import everygrams
    from collections import Counter
    # Get files
    fileinfo = []
    for root, dirs, files in os.walk(DATAPATH):
        for name in files:
            if name.endswith(SURFIX):
                date = name.replace(SURFIX, '')
                furi = (os.path.join(root, name))
                fileinfo.append({'date':date, 'uri': furi})
    fileinfo = pd.DataFrame(fileinfo)
    # Perform ngram computing and merge
    wc = compute_freq(fileinfo['uri'].iloc[0], min_n=min_n, max_n=max_n)
    for i in range(1,fileinfo.shape[0]):
        tmp = compute_freq(fileinfo['uri'].iloc[i], min_n=min_n, max_n=max_n)
        wc = wc.copy() + tmp
    # Remove terms with length less than min_mlength
    if min_count>1:
        for k in wc.copy().keys():
            if wc[k]<min_count:
                del wc[k]
    #
    return(wc)


dic = count_ngrams('../data/cwb/weather_report', min_count=100)
print(dic)

Counter({'海面': 759, '風力': 534, '平均': 532, '均風': 528, '均風力': 507, '平均風': 505, '平均風力': 484, '陣風': 460, '臺灣': 457, '最大': 447, '大陣': 430, '大陣風': 410, '最大陣': 409, '最大陣風': 389, '北部': 385, '南部': 350, '注意': 334, '力將': 330, '風力將': 321, '均風力將': 306, '平均風力將': 291, '船隻': 289, '隻請': 281, '船隻請': 273, '面平': 271, '海面平': 263, '面平均': 256, '地區': 249, '海面平均': 248, '部海': 244, '面平均風': 242, '部海面': 236, '海面平均風': 234, '增強': 233, '面平均風力': 227, '海面平均風力': 219, '面及': 216, '東北': 214, '強至': 214, '海峽': 209, '海面及': 207, '增強至': 204, '部平': 188, '沙島': 188, '公里': 187, '東海': 186, '部平均': 182, '請注': 181, '島海': 178, '將增': 178, '部平均風': 175, '請注意': 174, '沙島海': 173, '影響': 172, '雷雨': 172, '島海面': 170, '部平均風力': 170, '將增強': 169, '力將增': 168, '沙島海面': 165, '風力將增': 164, '灣東': 163, '局部': 162, '有局': 161, '將增強至': 161, '力將增強': 159, '北部海': 158, '臺灣東': 157, '短暫': 155, '風力將增強': 155, '有局部': 154, '均風力將增': 153, '北部海面': 152, '別注': 151, '力將增強至': 151, '東南': 151, '特別': 149, '風力將增強至': 147, '別注意': 146, '面平均風力將': 146, '特別注': 144, '平均風力將增': 144, '均風力將增強'

從上面的結果可以看到，n-gram approach 雖然可以不需要預設的辭典，但是相對的「排除清單」會比斷詞的版本長很多。在我們的問題情境中，由於大部分的專有名詞已經事先知道，或許直接使用自定一辭典比較快速，但如果進行分析的目的是探索一個新的語料來源，n-gram approach 不失為一個初步的工具。

In [30]:
# 試試看大量文本的結果
wc = count_ngrams('D:/data/cwb_report', min_count=1000)
print(wc)

Counter({'海面': 36478, '平均': 24909, '均風': 24866, '風力': 24684, '平均風': 24049, '均風力': 23816, '平均風力': 22999, '最大': 21853, '陣風': 21742, '臺灣': 21398, '大陣': 20737, '最大陣': 19880, '大陣風': 19684, '最大陣風': 18827, '北部': 18630, '注意': 16769, '南部': 16212, '力將': 15452, '風力將': 14833, '船隻': 14542, '均風力將': 14263, '隻請': 14074, '平均風力將': 13632, '船隻請': 13612, '地區': 12647, '面平': 12185, '增強': 11847, '海面平': 11818, '面平均': 11737, '海面平均': 11370, '面平均風': 11244, '東北': 11227, '部海': 11128, '面及': 10895, '海面平均風': 10877, '部海面': 10778, '面平均風力': 10684, '海面及': 10494, '強至': 10320, '海面平均風力': 10317, '公里': 10200, '增強至': 9916, '請注': 9647, '海峽': 9552, '請注意': 9328, '東海': 8973, '影響': 8785, '將增': 8630, '短暫': 8302, '將增強': 8279, '雷雨': 8242, '部平': 8232, '局部': 7993, '沙島': 7986, '部平均': 7972, '力將增': 7945, '將增強至': 7908, '島海': 7905, '有局': 7803, '部平均風': 7727, '島海面': 7686, '力將增強': 7609, '沙島海': 7602, '風力將增': 7595, '部地': 7541, '天氣': 7533, '有局部': 7519, '沙島海面': 7383, '灣東': 7375, '部平均風力': 7362, '及東': 7343, '部地區': 7289, '北部海': 7275, '風力將增強': 7259, '均風